In [1]:
%load_ext autoreload
%autoreload 1
%aimport ecg_get_data
%aimport Models
%aimport train_test_validat
%aimport self_attention
%aimport ECGplot
import torch
import torchvision
from torch.autograd import Variable
from torchvision import datasets, transforms

import Models 
from train_test_validat import *
from self_attention import *
from  ecg_get_data import *

import matplotlib.pyplot as plt
import random
import time

random_seed = 2
torch.manual_seed(random_seed)    # reproducible
torch.cuda.manual_seed_all(random_seed)
random.seed(random_seed)
np.random.seed(random_seed)

time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime()) 

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_path =  './PNG_Power/' #路径
lable_path = './label.npy'
model_path = './model/'+time_str
log_path = './log/'+  time_str


EcgChannles_num = 12
EcgLength_num = 5000
DEVICE = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda:1


In [3]:
# -*- coding: utf-8 -*-
import numpy as np
import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data.dataloader import DataLoader
from torchvision.transforms import transforms
from pathlib import Path

from PIL import Image

class MyDataset(Dataset):
    def __init__(self, root, transform=None):
        self.transform = transforms.Compose([
            transforms.Resize([224, 224]),
            #transforms.Resize(224),
            #transforms.RandomResizedCrop((224,224)),
            transforms.ToTensor(), # 将图片(Image)转成Tensor，归一化至[0, 1]（直接除以255）
            transforms.Normalize(mean=[.5, .5, .5], std=[.5, .5, .5]) # 标准化至[-1, 1]，规定均值和标准差
            #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # 常用标准化

        ])
        images_path = Path(root)
        images_list = list(images_path.glob('*.png')) # list(images_path.glob('*.png'))
        
        images_list_str = [ str(x) for x in images_list ]
        #print(images_list_str)
        images_list_str.sort(key=lambda x:int(((x.split('.')[0])).split('/')[-1])) #把'root/xx.png' 先按“.”分割，再按“/”分割，得到编号xx，转为整形并排序      #排序
        self.images = images_list_str
        #print(images_list_str)
    def __getitem__(self, item):
        image_path = self.images[item]
        #print(image_path)
        # image = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), 1) # 读到的是BGR数据
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # 转化为RGB
        # # 这时的image是H, W, C的顺序，因此下面需要转化为：C, H, W
        # image = torch.from_numpy(image).permute(2, 0, 1)/255 # 归一化[0, 1]才能与PIL读取的数据一致
        image_num = int(((image_path.split('.')[0])).split('/')[-1])-1  #把'root/xx.png' 先按“.”分割，再按“/”分割，得到编号xx，转为整形并排序      #排序
        #print(image_num)
        image = Image.open(image_path)  # 读取到的是RGBA
        #print(image.format, image.size, image.mode)#打印出原图格式
        image = image.convert("RGB") 
        #print(image.format, image.size, image.mode)#打印出现在图格式
        image = self.transform(image)   # transform转化image为：C, H, W
        label = 1 if image_num<689 else 0 # 这是一个label的示例，可自定义
        return image, label

    def __len__(self):
        return len(self.images)
    def get_size(self):
        image_path = self.images[0]
        image = Image.open(image_path)
        image = image.convert("RGB") 
        image = self.transform(image)
        return image.size()

In [4]:
train_data = MyDataset(os.path.join(data_path,'train/'))
validate_date = MyDataset(os.path.join(data_path,'valid/'))

In [5]:
validate_date.__len__()

0

In [6]:
train_data.__len__()

0

In [7]:
train_data.get_size()

IndexError: list index out of range

In [8]:
import torch
from torch import nn
class Bottleneck(nn.Module):
    #每个stage维度中扩展的倍数
    extention=4
    def __init__(self,inplanes,planes,stride,downsample=None):
        '''

        :param inplanes: 输入block的之前的通道数
        :param planes: 在block中间处理的时候的通道数
                planes*self.extention:输出的维度
        :param stride:
        :param downsample:
        '''
        super(Bottleneck, self).__init__()

        self.conv1=nn.Conv2d(inplanes,planes,kernel_size=1,stride=stride,bias=False)
        self.bn1=nn.BatchNorm2d(planes)

        self.conv2=nn.Conv2d(planes,planes,kernel_size=3,stride=1,padding=1,bias=False)
        self.bn2=nn.BatchNorm2d(planes)

        self.conv3=nn.Conv2d(planes,planes*self.extention,kernel_size=1,stride=1,bias=False)
        self.bn3=nn.BatchNorm2d(planes*self.extention)

        self.relu=nn.ReLU(inplace=True)

        #判断残差有没有卷积
        self.downsample=downsample
        self.stride=stride

    def forward(self,x):
        #参差数据
        residual=x

        #卷积操作
        out=self.conv1(x)
        out=self.bn1(out)
        out=self.relu(out)

        out=self.conv2(out)
        out=self.bn2(out)
        out=self.relu(out)

        out=self.conv3(out)
        out=self.bn3(out)
        out=self.relu(out)

        #是否直连（如果Indentity blobk就是直连；如果Conv2 Block就需要对残差边就行卷积，改变通道数和size
        if self.downsample is not None:
            residual=self.downsample(x)

        #将残差部分和卷积部分相加
        out=residual+out
        out=self.relu(out)

        return out


class ResNet(nn.Module):
    def __init__(self,block,layers,num_class):
        #inplane=当前的fm的通道数
        self.inplane=64
        super(ResNet, self).__init__()

        #参数
        self.block=block
        self.layers=layers

        #stem的网络层
        self.conv1=nn.Conv2d(3,self.inplane,kernel_size=7,stride=2,padding=3,bias=False)
        self.bn1=nn.BatchNorm2d(self.inplane)
        self.relu=nn.ReLU()
        self.maxpool=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)

        #64,128,256,512指的是扩大4倍之前的维度，即Identity Block中间的维度
        self.stage1=self.make_layer(self.block,64,layers[0],stride=1)
        self.stage2=self.make_layer(self.block,128,layers[1],stride=2)
        self.stage3=self.make_layer(self.block,256,layers[2],stride=2)
        self.stage4=self.make_layer(self.block,512,layers[3],stride=2)

        #后续的网络
        self.avgpool=nn.AvgPool2d(7)
        self.fc=nn.Linear(512*block.extention,num_class)
        self.softmax = nn.Softmax(dim=-1)
        self.dropout = torch.nn.Dropout(0.5)
    def forward(self,x):
        #stem部分：conv+bn+maxpool
        out=self.conv1(x)
        out=self.bn1(out)
        out=self.relu(out)
        out=self.maxpool(out)
        #print(out.shape)

        #block部分
        out=self.stage1(out)
        #print(out.shape)
        out=self.stage2(out)
        #print(out.shape)
        out=self.stage3(out)
        #print(out.shape)
        out=self.dropout(self.stage4(out))
        #print(out.shape)
        #分类
        out=self.avgpool(out)
        #print(out.shape)
        out=torch.flatten(out,1)
        #print(out.shape)
        out=self.fc(out)
        out=self.softmax(out)
        return out

    def make_layer(self,block,plane,block_num,stride=1):
        '''
        :param block: block模板
        :param plane: 每个模块中间运算的维度，一般等于输出维度/4
        :param block_num: 重复次数
        :param stride: 步长
        :return:
        '''
        block_list=[]
        #先计算要不要加downsample
        downsample=None
        if(stride!=1 or self.inplane!=plane*block.extention):
            downsample=nn.Sequential(
                nn.Conv2d(self.inplane,plane*block.extention,stride=stride,kernel_size=1,bias=False),
                nn.BatchNorm2d(plane*block.extention)
            )

        # Conv Block输入和输出的维度（通道数和size）是不一样的，所以不能连续串联，他的作用是改变网络的维度
        # Identity Block 输入维度和输出（通道数和size）相同，可以直接串联，用于加深网络
        #Conv_block
        conv_block=block(self.inplane,plane,stride=stride,downsample=downsample)
        block_list.append(conv_block)
        self.inplane=plane*block.extention

        #Identity Block
        for i in range(1,block_num):
            block_list.append(block(self.inplane,plane,stride=1))

        return nn.Sequential(*block_list)

In [9]:
BATCH_SIZE = 32
FOLDS = 1
EPOCHS = 5000  
PATIENCE = 36
LR = 0.001

In [10]:
from torch.utils.tensorboard import SummaryWriter   
os.makedirs(model_path, exist_ok=True)
writer = SummaryWriter(log_path)
from torchsummary import summary

In [11]:
NET = [ResNet(Bottleneck,[3,4,6,3],2) for i in range(FOLDS)]
print(summary(NET[0], (3, 224, 224), device="cpu"))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
             ReLU-13          [-1, 256, 56, 56]               0
           Conv2d-14          [-1, 256,

In [12]:
from torch.optim.lr_scheduler import CosineAnnealingLR

for fold in range(FOLDS):
    
    early_stopping = EarlyStopping(PATIENCE, verbose=True, model_path=model_path, delta=0)
    #train_dataset,valid_dataset = get_k_fold_dataset(fold=int(fold+1),x = train_x,y=train_y,k=FOLDS,random_seed = random_seed)
    train_dataloader = Data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
    valid_dataloader = Data.DataLoader(dataset=validate_date, batch_size=BATCH_SIZE, shuffle=True)

    NET[fold].to(DEVICE)
    optimizer  = torch.optim.Adam(NET[fold].parameters(), lr=LR,weight_decay=1e-4)  
    criterion = torch.nn.CrossEntropyLoss()   
    #等间隔调整学习率
    scheduler =torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1, last_epoch=-1)
    #scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max = 32)
    best_loss = 3
    for epoch in range(1,EPOCHS):
        time_all=0
        start_time = time.time()
        train_loss,train_acc = train_model(train_dataloader, NET[fold], criterion, optimizer,DEVICE) # 训练模型
        

        time_all = time.time()-start_time
        validate_loss,validate_acc = test_model(valid_dataloader,criterion,NET[fold],DEVICE) # 测试模型
        writer.add_scalars(main_tag=str(fold)+'_Loss',tag_scalar_dict={'train': train_loss,'validate': validate_loss},global_step=epoch)
        writer.add_scalars(main_tag=str(fold)+'_Accuracy',tag_scalar_dict={'train': train_acc,'validate': validate_acc},global_step=epoch)
        writer.add_scalars(main_tag=str(fold)+'_LearningRate',tag_scalar_dict={'LR': optimizer.state_dict()['param_groups'][0]['lr']},global_step=epoch)

        print('- Epoch: %d - Train_loss: %.5f - Train_acc: %.5f - Val_loss: %.5f - Val_acc: %5f - T_Time: %.3f' %(epoch,train_loss,train_acc,validate_loss,validate_acc,time_all))
        print('当前学习率：%f' %optimizer.state_dict()['param_groups'][0]['lr'])

        if validate_loss < best_loss:
            best_loss = validate_loss
            print('Find better model in Epoch {0}, saving model.'.format(epoch))
            #torch.save(NET[fold],  model_path+'/best_model_' + str(fold) + '.pt')  # 保存最优模型
        else:
            scheduler.step() # 学习率迭代
        #是否满足早停法条件
        if(early_stopping(validate_loss,NET[fold])):
            print("Early stopping")
            break

    print('Fold %d Training Finished' %(fold+1))
    torch.cuda.empty_cache()# 清空显卡cuda
print('Training Finished')

- Epoch: 1 - Train_loss: 0.70071 - Train_acc: 0.59784 - Val_loss: 0.71272 - Val_acc: 0.602679 - T_Time: 42.296
当前学习率：0.001000
Find better model in Epoch 1, saving model.
Validation loss decreased (inf --> 0.712720).  Saving model ...
                    --------------------------------------------------

- Epoch: 2 - Train_loss: 0.62686 - Train_acc: 0.67354 - Val_loss: 0.74408 - Val_acc: 0.549107 - T_Time: 40.325
当前学习率：0.001000
EarlyStopping counter: 1 out of 36

- Epoch: 3 - Train_loss: 0.64608 - Train_acc: 0.64750 - Val_loss: 0.75967 - Val_acc: 0.553571 - T_Time: 40.270
当前学习率：0.001000
EarlyStopping counter: 2 out of 36

- Epoch: 4 - Train_loss: 0.71237 - Train_acc: 0.60050 - Val_loss: 0.77308 - Val_acc: 0.540179 - T_Time: 40.674
当前学习率：0.001000
EarlyStopping counter: 3 out of 36

- Epoch: 5 - Train_loss: 0.71256 - Train_acc: 0.60071 - Val_loss: 0.79987 - Val_acc: 0.513393 - T_Time: 40.460
当前学习率：0.001000
EarlyStopping counter: 4 out of 36

- Epoch: 6 - Train_loss: 0.71240 - Train_acc: 